In [1]:
#---------------------------------------------------------------------------------------------GENERAL-----------------------------------------------------------------------------------------------
import torch
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.datasets import VOCDetection
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from torchvision.transforms import Compose, Resize, ToTensor
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
import torch.multiprocessing as mp
from torchvision.transforms import functional as F
from torchvision.models.detection.faster_rcnn import FasterRCNN_ResNet50_FPN_Weights
from torch.cuda.amp import autocast, GradScaler
from sklearn.metrics import average_precision_score
from sklearn.metrics import precision_recall_fscore_support
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
from typing import Dict
import numpy as np
from torch.utils.data import Dataset
import os
from PIL import Image
import xml.etree.ElementTree as ET
import collections
from torchvision import transforms
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import torch.nn as nn
import torchviz
from torchsummary import summary
import pytesseract
import re
import cv2  
import easyocr
import keras_ocr
import math
from datetime import datetime
from collections import defaultdict


# Specify Tesseract path (example for Windows)
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

#-------------------------------------------------------------------------------------JUPYTER NOTEBOOK SETTINGS-------------------------------------------------------------------------------------
from IPython.core.display import display, HTML                                    
display(HTML("<style>.container { width:100% !important; }</style>"))     

C:\Users\Ciprian-Florin Ifrim\AppData\Local\Temp\ipykernel_13628\1770188872.py:50: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
# Load the pretrained model
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT)

# Replace the classifier with a new one
num_classes = 6
in_features = model.roi_heads.box_predictor.cls_score.in_features

model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

# Put model to device
device = torch.device('cuda')
model.to(device)

# Load saved model
model.load_state_dict(torch.load('pytorch_rcnn_models/FINAL-v2_rcnn_batch-16_epoch-40_full-enchanced-original_non-augmented.pt'))
model.eval()

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

In [3]:
# Data processing
data_transforms = transforms.Compose([
    transforms.Resize(512),                                            
    transforms.ToTensor(), 
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:
img_id = input("Please enter a number: ")
image_path = 'pascal_voc_datasets/VOCdevkit/PlotsEnchanced_Original_NoAugmentation/JPEGImages/'
img_name = 'image'+ img_id 
CUSTOM_CLASSES = {"name": 1, "value": 2, "x-axis": 3, "y-axis": 4, "plot":5}

img = Image.open(image_path + img_name + '.jpg')
orig_width, orig_height = img.size                                   # Store the original image dimensions

# Process the image
img_processed = data_transforms(img)

# Calculate the scale ratios
new_width, new_height = img_processed.shape[1], img_processed.shape[2]

# Calculate the scale ratios
if orig_width <= orig_height:
    new_height = 512
    new_width = int(orig_width * (new_height / orig_height))
else:
    new_width = 512
    new_height = int(orig_height * (new_width / orig_width))
    
width_ratio = new_width / orig_width
height_ratio = new_height / orig_height

img_processed = img_processed.to(device)
with torch.no_grad():
    prediction = model([img_processed])

# Scale bounding boxes back to original image size
for box in prediction[0]['boxes']:
    box[0] *= (1 / width_ratio)
    box[2] *= (1 / width_ratio)
    box[1] *= (1 / height_ratio)
    box[3] *= (1 / height_ratio)

# Debugging
#print(prediction[0]['boxes'])
print(prediction[0]['scores'])


results = {}                                 # Dictionary to store the results
yaxis_coordinates = []                       # Initialize a list to store the Y coordinates
label5_image = None                          # Variable to store the cropped image for label 5
y_axis_image = None

reader = easyocr.Reader(lang_list=['en'])       # Create a reader to recognize the English language

for box, label, score in zip(prediction[0]['boxes'], prediction[0]['labels'], prediction[0]['scores']):
    if label >= 1 and label <= 4 and score > 0.6:                                                 # Check if the score is above 0.7 and label is between 1 and 4
        xmin, ymin, xmax, ymax = [int(i) for i in box]                                            # Convert box coordinates to integers
        cropped_image = img.crop((xmin, ymin, xmax, ymax))                                        # Crop the image          
        bw_img = cropped_image.convert("L")                                                       # Convert the cropped image to grayscale (black and white)
        bw_img_array = np.array(bw_img)

        # Resize the image (e.g., double the size)
        resized_img = cv2.resize(bw_img_array, (0,0), fx=10, fy=10, interpolation = cv2.INTER_CUBIC)

        # Contrast Enhancement
        clahe = cv2.createCLAHE(clipLimit=4, tileGridSize=(16, 16))
        enhanced_img = clahe.apply(resized_img)

        # Apply Otsu's thresholding
        _, thresh_img = cv2.threshold(enhanced_img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        
        plt.figure(figsize=(10, 10))
        plt.imshow(thresh_img, cmap='gray')
        plt.title('thresh_img')
        plt.axis('off')  # To turn off the axis
        plt.show()
        
        if label == CUSTOM_CLASSES["y-axis"]:
            y_axis_image = cropped_image
            
            result = reader.readtext(thresh_img)
            text_results = [item[1] for item in result]                            # Extract only the text
            text_found = ','.join(text_results)
            
            for (bbox, text, prob) in result:
                bottom_y1 = bbox[1][1]  # Get the y-coordinate of one of the bottom points
                bottom_y2 = bbox[2][1]  # Get the y-coordinate of the other bottom point
                #print(bottom_y1, bottom_y2)
                average_y = (bottom_y1 + bottom_y2) / 2  # Calculate the average y-coordinate
                quarter_y = (bottom_y2 + average_y) / 2
                yaxis_coordinates.append(int(quarter_y / 10))  # Add the average y-coordinate for each text box available (divide by the upscale factor)
        else:
            result = reader.readtext(thresh_img, detail = 0)      
            text_found = ','.join(result)
            text_found = re.sub('\n+', ',', text_found)                                           # Replace one or more consecutive newline characters with a single comma
            
                                                                  
        label_name = [name for name, id in CUSTOM_CLASSES.items() if id == label][0]              # Find the label name using the CUSTOM_CLASSES dictionary
        results[label_name] = text_found                                                          # Store the result in the dictionary
        
    elif label == 5 and score > 0.7:                                                              # Check if the label is 5 and the score is above 0.7
        xmin, ymin, xmax, ymax = [int(i) for i in box]                                            # Convert box coordinates to integers
        label5_image = img.crop((xmin, ymin, xmax, ymax))                                         # Crop the image for label 5

# The results dictionary now contains the extracted text for labels 1 to 4
print(results)

#############################################################################################################################
plt.imshow(label5_image)
plt.title("original plot")
plt.axis('off')
plt.show()
    
# Plot processing
bw_label5_img = label5_image.convert("L")                                                       # Convert the cropped image to grayscale (black and white)
bw_label5_img_array = np.array(bw_label5_img)

# Apply Gaussian blur to reduce noise and improve the result of Canny edge detection
blurred_label5_img = cv2.GaussianBlur(bw_label5_img_array, (3, 3), 0)

# First, apply a binary threshold on the image
thresh_label5_img = cv2.adaptiveThreshold(blurred_label5_img, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 11, 2)
inverted_label5_img = cv2.bitwise_not(thresh_label5_img)

# Define a kernel for the erosion (a 3x3 square is common)
kernel = np.ones((2,2), np.uint8)
closed_label5_img = cv2.morphologyEx(inverted_label5_img, cv2.MORPH_CLOSE, kernel)

plt.imshow(closed_label5_img, cmap='gray')
plt.title('Postprocessed Image')
plt.axis('off')
plt.show()

######################################################################################################################
def filter_coordinates(pixel_coordinates, threshold=100):
    horizontal_lines = {}
    vertical_lines = {}

    # Check the shape of the input and reshape if needed
    if pixel_coordinates.shape[1] == 1 and pixel_coordinates.shape[2] == 2:
        pixel_coordinates = pixel_coordinates.reshape(-1, 2)

    # Separate coordinates into horizontal and vertical lines
    for coord in pixel_coordinates:
        x, y = int(coord[0]), int(coord[1])
        if y not in horizontal_lines:
            horizontal_lines[y] = []
        if x not in vertical_lines:
            vertical_lines[x] = []
        horizontal_lines[y].append(x)
        vertical_lines[x].append(y)

    # Filter horizontal lines longer than the threshold
    for y, x_values in horizontal_lines.items():
        if len(x_values) > threshold:
            pixel_coordinates = pixel_coordinates[np.logical_not(np.isin(pixel_coordinates[:, 0], x_values)), :]

    # Filter vertical lines longer than the threshold
    for x, y_values in vertical_lines.items():
        if len(y_values) > threshold:
            pixel_coordinates = pixel_coordinates[np.logical_not(np.isin(pixel_coordinates[:, 1], y_values)), :]

    return pixel_coordinates

def remove_corner_pixels(coordinates, width, height):
    # Define a function to count pixels near corners
    def count_corner_pixels(coordinates, distance):
        count = 0
        for coord in coordinates:
            x, y = coord[0], coord[1]
            if ((x < distance or x >= width - distance) and (y < distance or y >= height - distance)):
                count += 1
        return count

    # Start with an initial distance
    distance = 10
    corner_pixels_count = count_corner_pixels(coordinates, distance)

    # Adjust the distance based on the count of pixels near corners
    if corner_pixels_count != 0:
        distance = 100 / corner_pixels_count

    filtered_coords = []
    for coord in coordinates:
        x, y = coord[0], coord[1]
        # Check if the pixel is in or within the calculated distance of the corners
        if ((x < distance or x >= width - distance) and (y < distance or y >= height - distance)):
            continue
        filtered_coords.append(coord)
        
    return np.array(filtered_coords)

def draw_pixels(image, coordinates, color=255, thickness=1):
    for coord in coordinates:
        x, y = int(coord[0]), int(coord[1])
        image[max(y - thickness, 0):min(y + thickness + 1, image.shape[0]),
              max(x - thickness, 0):min(x + thickness + 1, image.shape[1])] = color
    return image

# Find the contours
contours, hierarchy = cv2.findContours(closed_label5_img, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

# Calculate the perimeter for each contour and find the index of the longest one
max_length = 0
longest_contour_index = 0
for i, contour in enumerate(contours):
    length = cv2.arcLength(contour, True) # The 'True' argument indicates that the contour is closed
    if length > max_length:
        max_length = length
        longest_contour_index = i
        
# Extract the longest contour
longest_contour = contours[longest_contour_index]
        
# Create a blank image to draw the longest contour
output_image = np.zeros_like(closed_label5_img)

# Draw the longest contour on the output image
cv2.drawContours(output_image, [longest_contour], -1, (255, 255, 255), 2)

# Show the image with the drawn contour using Matplotlib
plt.imshow(output_image, cmap='gray')
plt.title('Longest Contour')
plt.axis('off')
plt.show()

output_image2 = np.zeros_like(closed_label5_img)
height, width = output_image2.shape
filtered_coordinates = filter_coordinates(longest_contour)
filtered_coordinates = remove_corner_pixels(filtered_coordinates, width, height)

output_image2 = draw_pixels(output_image2, filtered_coordinates)

plt.imshow(output_image2, cmap='gray')
plt.title('Longest Contour2')
plt.axis('off')
plt.show()

# Finding the highest pixel (smallest y value)
highest_pixel = filtered_coordinates[filtered_coordinates[:, 1].argmin()]

# Finding the lowest pixel (largest y value)
lowest_pixel = filtered_coordinates[filtered_coordinates[:, 1].argmax()]

# Finding the rightmost pixel (largest x value), and among those, the one with the lowest y value
right_most_pixels = filtered_coordinates[filtered_coordinates[:, 0] == filtered_coordinates[:, 0].max()]
right_most_pixel = right_most_pixels[right_most_pixels[:, 1].argmin()]



print("Highest pixel:", highest_pixel)
print("Lowest pixel:", lowest_pixel)
print("Rightmost pixel:", right_most_pixel)

new_pixels = [highest_pixel, lowest_pixel, right_most_pixel]
output_image3 = np.zeros_like(closed_label5_img)
output_image3 = draw_pixels(output_image3, new_pixels)

plt.imshow(output_image3, cmap='gray')
plt.title('Longest Contour2')
plt.axis('off')
plt.show()

######################################################################################################################
# this function finds the average difference between all the elements in a list, then adds a new element at the end
# equal to the last element minus the average
def calculate_new_list(lst):
    # Calculate the differences between successive elements
    differences = [lst[i] - lst[i + 1] for i in range(len(lst) - 1)]
    
    # Calculate the average difference
    avg_difference = sum(differences) / len(differences)
    
    # Add a new element at the end of the list
    new_element = lst[-1] - avg_difference 
    lst.append(new_element)

    return lst

def extract_currency(string):
    # Dictionary to map symbols to currency names
    currency_map = {
        'US': 'USD',
        '$': 'USD',
        '€': 'EUR',
        '£': 'GBP',
        '¥': 'JPY',
        'A$': 'AUD',
        'C$': 'CAD',
        'S': 'USD',   # Misidentification for $
        'E': 'EUR',   # Misidentification for €
        'L': 'GBP',   # Misidentification for £
        'Y': 'JPY',   # Potential misidentification for ¥
    }
    
    # Build a pattern that checks for each symbol followed by optional whitespace and a digit or a period
    pattern_elements = [re.escape(symbol) + r'\s*[0-9\.]' for symbol in currency_map.keys()]
    pattern_string = '|'.join(pattern_elements) + r'|USD|EUR|GBP|JPY|AUD|CAD'
    pattern = re.compile(pattern_string)
    
    match = pattern.search(string)
    
    if match:
        symbol = match.group()[:-1].strip()  # Exclude the trailing digit or period and any whitespace from the match
        return currency_map.get(symbol, symbol).lower()
    else:
        return None

def extract_percentage(string):
    match = re.search(r'([-+]?\d+(?:\.\d+)?)\s*%', string)
    if match:
        raw_value = correct_decimal_point(match.group(1))
        percentage = float(raw_value)
        return percentage
    return None

def extract_bracketed_timeframe(string):
    match = re.search(r'\(([^)]+)\)', string)
    if match:
        bracketed_string = match.group(1)
        return bracketed_string.lower()
    return None

def extract_float_value(input_string):
    numbers = re.findall(r'\d+\.\d+', input_string)                                      # Find all the floating-point numbers in the string using a regular expression
    return float(numbers[0])                                                             # Return the first floating-point number found, as a float
    
def correct_misidentifications(text):
    corrections = {
        'O': '0',
        'G': '6',
        'B': '8',
        'I': '1',
        'S': '5',
        'Z': '2',
        'Q': '0',
        'T': '7',
        'L': '1',
        'A': '4'
    }
    
    corrected_text = text
    for char, replacement in corrections.items():
        corrected_text = corrected_text.replace(char, replacement)
    
    corrected_text = re.sub(r'[^\d.]+', '', corrected_text)  # Keep only numeric characters and decimal points
    return corrected_text

def correct_decimal_point(value):
    value_str = str(value)

    # Check if the first digit is '0' and the second character is not '.'
    if value_str[0] == '0' and (len(value_str) == 1 or value_str[1] != '.'):
        value_str = '0.' + value_str[1:]

    # If the original value was a number, return as a float; otherwise, return as a string
    if isinstance(value, (int, float)):
        return float(value_str)
    return value_str

def process_ocr_string(y_axis):
    values = y_axis.split(',')
    corrected_values = []
    removed_indices = []
    
    for index, value in enumerate(values):
        # If the value contains a space not at the beginning or end, replace with a decimal point
        value = re.sub(r'(?<=\S) (?=\S)', '.', value)
        
        # If the value contains both digits and characters
        if re.search(r'\d', value) and re.search(r'[a-zA-Z]', value):
            corrected_value = correct_misidentifications(value)
        else:
            corrected_value = value

        # If the corrected value is still a valid float, add it to the results
        try:
            corrected_values.append(float(correct_decimal_point(corrected_value)))
        except ValueError:
            removed_indices.append(index)  # Save the index of the value that was removed

    return corrected_values, set(removed_indices)

def time_mapping(input_string):
    mapping = {
        '1h': '1 hour',
        '4h': '4 hours',
        '24h': '1 day',
        '1d': '1 day',
        '7d': '7 days',
        '1w': '7 days',
        '1m': '1 month',
        '3m': '3 months',
        '6m': '6 months',
        '1y': '1 year',
        'ytd': 'year to date',
        'all': 'year to date'
    }
    return mapping.get(input_string, 'N/A')

def correct_value(text):
    if text and text[0].isdigit() and text[0] in ['S', '5']:
        return '$' + text[1:]
    return text
######################################################################################################################
# DRAWING THE Y-AXIS DATA LOCATION ON THE IMAGE
# Apply the function to both lists
y_axis_str = results['y-axis']
y_axis_values, y_axis_val_rem_idx = process_ocr_string(y_axis_str)
yaxis_coordinates = [value for index, value in enumerate(yaxis_coordinates) if index not in y_axis_val_rem_idx]
print(y_axis_values, len(y_axis_values))
print(yaxis_coordinates, len(yaxis_coordinates))
print(y_axis_val_rem_idx)
y_axis_values_new = calculate_new_list(y_axis_values)
yaxis_coordinates_new = calculate_new_list(yaxis_coordinates)
yaxis_coordinates_new = list(map(round, yaxis_coordinates_new))

print("New Y-axis values:", y_axis_values_new)
print("New Y coordinates:", yaxis_coordinates_new)

y_axis_image = np.array(y_axis_image)
for i,item in enumerate(yaxis_coordinates_new):
    # Draw the identified pixels
    x, y = 50, int(item)
    cv2.circle(y_axis_image, (x, y), radius=3, color=255, thickness=-1)

# Show the image with the drawn pixels using Matplotlib
plt.figure(figsize=(10, 10))
plt.imshow(y_axis_image)
plt.title('Identified Y')
plt.axis('off')
plt.show()

######################################################################################################################
# FIRST INTERPOLATION METHOD
# Value interpolation from curve
from scipy.interpolate import interp1d

# Create a linear interpolation function
f = interp1d(yaxis_coordinates_new, y_axis_values_new)

# Example usage
type1_interp_current_val = round(float(f(right_most_pixel[1])), 5)
print(f"The Y-axis value for the Y coordinate {right_most_pixel[1]} is {type1_interp_current_val}")

# value from plot
value_str = correct_value(correct_decimal_point(results['value']))
stock_value = extract_float_value(value_str)
print(f"The stock value from the plot is {stock_value}")

# value difference/accuracy
type1_pct_diff = abs(round(((stock_value - type1_interp_current_val) / stock_value) * 100, 5))
print(f"The percentage difference is {type1_pct_diff:.2f}%")

######################################################################################################################
# SECOND INTERPOLATION METHOD
y_axis_val_diff = abs(y_axis_values[-1] - y_axis_values[0])
y_axis_coord_dif = abs(yaxis_coordinates[-1] - yaxis_coordinates[0])
y_axis_val_per_pixel = y_axis_val_diff/y_axis_coord_dif

print(f"The calculated value per pixel for the 2nd interpolation method is: {y_axis_val_per_pixel}")

plot_height = label5_image.height
type2_interp_current_val = round((abs(plot_height - right_most_pixel[1])) * y_axis_val_per_pixel, 5)

print(f"Curve value based on secondaly calculation is {type2_interp_current_val}")

# value from plot
type2_pct_diff = abs(round(((stock_value - type2_interp_current_val) / stock_value) * 100, 3))
print(f"The percentage difference is {type2_pct_diff:.3f}%")
######################################################################################################################
# Compare the 2 methods and choose the one with the lowest percentage difference
if type1_pct_diff < type2_pct_diff:
    lowest_value  = round(float(f(lowest_pixel[1])), 5)
    highest_value = round(float(f(highest_pixel[1])), 5)
else:
    lowest_value  = round((abs(plot_height - lowest_pixel[1])) * y_axis_val_per_pixel, 5)
    highest_value = round((abs(plot_height - highest_pixel[1])) * y_axis_val_per_pixel, 5)

print(f"The lowest value is: {lowest_value}")
print(f"The highest value is: {highest_value}")
######################################################################################################################
# Extracting all the other values
# Define the observations
currency = extract_currency(results['value'])
percentage_drop = extract_percentage(results['value'])
print(f"The raw OCR reading fro the currency is: {results['value']}, the corrected output is: {currency}")
print(f"The raw OCR reading for the percentage drop is: {results['value']}, the corrected output is: {percentage_drop}")
data = {
    'name': re.split(r',| Price', results['name'])[0].lower(),
    'currency': currency,
    'value.pct.drop': extract_percentage(results['value']),
    'current.value': stock_value,
    'type1.interp.current.value': type1_interp_current_val,
    'type2.interp.current.value': type2_interp_current_val,
    'type1.curr.val.pct.diff': type1_pct_diff,
    'type2.curr.val.pct.diff': type2_pct_diff,
    'lowest.value': lowest_value,
    'highest.value': highest_value,
    'timeframe': time_mapping(extract_bracketed_timeframe(results['value'])),
    'date': datetime.now().date(),
    'time': datetime.now().time().strftime('%H:%M:%S')                            # Format the time properly
}

# Create a DataFrame
df = pd.DataFrame(data, index = [0])
dataset_path = "datasets/dataset.csv"

# Create the directory if it doesn't exist
directory = os.path.dirname(dataset_path)
if not os.path.exists(directory):
    os.makedirs(directory)

# Save the DataFrame to the CSV file
df.to_csv(dataset_path, index=False)

In [19]:
    def calculate_new_list(lst):
        # Calculate the differences between successive elements
        differences = [lst[i] - lst[i + 1] for i in range(len(lst) - 1)]

        # Calculate the average difference
        avg_difference = sum(differences) / len(differences)

        # Add a new element at the end of the list
        new_element = lst[-1] - avg_difference 
        lst.append(new_element)

        return lst

    def extract_currency(string):
        # Dictionary to map symbols to currency names
        currency_map = {
            'US': 'USD',
            '$': 'USD',
            '€': 'EUR',
            '£': 'GBP',
            '¥': 'JPY',
            'A$': 'AUD',
            'C$': 'CAD',
            'S': 'USD',   # Misidentification for $
            'E': 'EUR',   # Misidentification for €
            'L': 'GBP',   # Misidentification for £
            'Y': 'JPY',   # Potential misidentification for ¥
        }

        # Build a pattern that checks for each symbol followed by optional whitespace and a digit or a period
        pattern_elements = [re.escape(symbol) + r'\s*[0-9\.]' for symbol in currency_map.keys()]
        pattern_string = '|'.join(pattern_elements) + r'|USD|EUR|GBP|JPY|AUD|CAD'
        pattern = re.compile(pattern_string)

        match = pattern.search(string)

        if match:
            symbol = match.group()[:-1].strip()  # Exclude the trailing digit or period and any whitespace from the match
            return currency_map.get(symbol, symbol).lower()
        else:
            return None

    def extract_percentage(string):
        match = re.search(r'([-+]?\d+(?:\.\d+)?)\s*%', string)
        if match:
            raw_value = correct_decimal_point(match.group(1))
            percentage = float(raw_value)
            return percentage
        return None

    def extract_bracketed_timeframe(string):
        match = re.search(r'\(([^)]+)\)', string)
        if match:
            bracketed_string = match.group(1)
            return bracketed_string.lower()
        return None

    def extract_float_value(input_string):
        numbers = re.findall(r'\d+\.\d+', input_string)                                      # Find all the floating-point numbers in the string using a regular expression
        return float(numbers[0])                                                             # Return the first floating-point number found, as a float

    def correct_misidentifications(text):
        corrections = {
            'O': '0',
            'G': '6',
            'B': '8',
            'I': '1',
            'S': '5',
            'Z': '2',
            'Q': '0',
            'T': '7',
            'L': '1',
            'A': '4'
        }

        corrected_text = text
        for char, replacement in corrections.items():
            corrected_text = corrected_text.replace(char, replacement)

        corrected_text = re.sub(r'[^\d.e-]+', '', corrected_text)  # Keep only numeric characters, decimal points, 'e', and '-'
        return corrected_text

    def process_ocr_string(y_axis):
        values = y_axis.split(',')
        corrected_values = []
        removed_indices = []

        for index, value in enumerate(values):
            # If the value contains a space not at the beginning or end, replace with a decimal point
            value = re.sub(r'(?<=\S) (?=\S)', '.', value)

            # If the value contains both digits and characters
            if re.search(r'\d', value) and re.search(r'[a-zA-Z]', value):
                corrected_value = correct_misidentifications(value)
            else:
                corrected_value = value

            # If the corrected value is still a valid float, add it to the results
            try:
                corrected_values.append(float(correct_decimal_point(corrected_value)))
            except ValueError:
                removed_indices.append(index)  # Save the index of the value that was removed

        return corrected_values, set(removed_indices)

    def correct_decimal_point(value):
        value_str = str(value)

        # Check if the value is in scientific notation
        if 'e' in value_str or 'E' in value_str:
            return float(value_str)

        # Check if the first digit is '0' and the second character is not '.'
        if value_str[0] == '0' and (len(value_str) == 1 or value_str[1] != '.'):
            value_str = '0.' + value_str[1:]

        # If the original value was a number, return as a float; otherwise, return as a string
        if isinstance(value, (int, float)):
            return float(value_str)
        return value_str

    def time_mapping(input_string):
        mapping = {
            '1h': '1 hour',
            '4h': '4 hours',
            '24h': '1 day',
            '1d': '1 day',
            '7d': '7 days',
            '1w': '7 days',
            '1m': '1 month',
            '3m': '3 months',
            '6m': '6 months',
            '1y': '1 year',
            'ytd': 'year to date',
            'all': 'year to date'
        }
        return mapping.get(input_string, 'N/A')

    def correct_value(text):
        if text and text[0].isdigit() and text[0] in ['S', '5']:
            return '$' + text[1:]
        return text
    
    ######################################################################################################################
    def filter_coordinates(pixel_coordinates, threshold=100):
        horizontal_lines = {}
        vertical_lines = {}

        # Check the shape of the input and reshape if needed
        if pixel_coordinates.shape[1] == 1 and pixel_coordinates.shape[2] == 2:
            pixel_coordinates = pixel_coordinates.reshape(-1, 2)

        # Separate coordinates into horizontal and vertical lines
        for coord in pixel_coordinates:
            x, y = int(coord[0]), int(coord[1])
            if y not in horizontal_lines:
                horizontal_lines[y] = []
            if x not in vertical_lines:
                vertical_lines[x] = []
            horizontal_lines[y].append(x)
            vertical_lines[x].append(y)

        # Filter horizontal lines longer than the threshold
        for y, x_values in horizontal_lines.items():
            if len(x_values) > threshold:
                pixel_coordinates = pixel_coordinates[np.logical_not(np.isin(pixel_coordinates[:, 0], x_values)), :]

        # Filter vertical lines longer than the threshold
        for x, y_values in vertical_lines.items():
            if len(y_values) > threshold:
                pixel_coordinates = pixel_coordinates[np.logical_not(np.isin(pixel_coordinates[:, 1], y_values)), :]

        return pixel_coordinates

    def remove_corner_pixels(coordinates, width, height):
        # Define a function to count pixels near corners
        def count_corner_pixels(coordinates, distance):
            count = 0
            for coord in coordinates:
                x, y = coord[0], coord[1]
                if ((x < distance or x >= width - distance) and (y < distance or y >= height - distance)):
                    count += 1
            return count

        # Start with an initial distance
        distance = 10
        corner_pixels_count = count_corner_pixels(coordinates, distance)

        # Adjust the distance based on the count of pixels near corners
        if corner_pixels_count != 0:
            distance = 100 / corner_pixels_count

        filtered_coords = []
        for coord in coordinates:
            x, y = coord[0], coord[1]
            # Check if the pixel is in or within the calculated distance of the corners
            if ((x < distance or x >= width - distance) and (y < distance or y >= height - distance)):
                continue
            filtered_coords.append(coord)

        return np.array(filtered_coords)

In [41]:
import os
import random

# Specify the directory path
directory_path = 'pascal_voc_datasets/VOCdevkit/PlotsEnchanced_Original_NoAugmentation/JPEGImages/'

# Get all filenames from the directory
filenames = [f for f in os.listdir(directory_path) if os.path.isfile(os.path.join(directory_path, f))]

# Shuffle the filenames
random.shuffle(filenames)

first_algo_diff = []
second_algo_diff = []
for i in tqdm(range(0, 1000), desc="Processing files"):
    img_name = filenames[i]
    CUSTOM_CLASSES = {"name": 1, "value": 2, "x-axis": 3, "y-axis": 4, "plot":5}

    img = Image.open(directory_path + img_name)
    orig_width, orig_height = img.size                                   # Store the original image dimensions

    # Process the image
    img_processed = data_transforms(img)

    # Calculate the scale ratios
    new_width, new_height = img_processed.shape[1], img_processed.shape[2]

    # Calculate the scale ratios
    if orig_width <= orig_height:
        new_height = 512
        new_width = int(orig_width * (new_height / orig_height))
    else:
        new_width = 512
        new_height = int(orig_height * (new_width / orig_width))

    width_ratio = new_width / orig_width
    height_ratio = new_height / orig_height
    
    img_processed = img_processed.to(device)
    with torch.no_grad():
        prediction = model([img_processed])

    # Scale bounding boxes back to original image size
    for box in prediction[0]['boxes']:
        box[0] *= (1 / width_ratio)
        box[2] *= (1 / width_ratio)
        box[1] *= (1 / height_ratio)
        box[3] *= (1 / height_ratio)


    results = {}                                 # Dictionary to store the results
    yaxis_coordinates = []                       # Initialize a list to store the Y coordinates
    label5_image = None                          # Variable to store the cropped image for label 5
    y_axis_image = None

    reader = easyocr.Reader(lang_list=['en'])       # Create a reader to recognize the English language

    for box, label, score in zip(prediction[0]['boxes'], prediction[0]['labels'], prediction[0]['scores']):
        if label >= 1 and label <= 4 and score > 0.6:                                                 # Check if the score is above 0.7 and label is between 1 and 4
            xmin, ymin, xmax, ymax = [int(i) for i in box]                                            # Convert box coordinates to integers
            cropped_image = img.crop((xmin, ymin, xmax, ymax))                                        # Crop the image          
            bw_img = cropped_image.convert("L")                                                       # Convert the cropped image to grayscale (black and white)
            bw_img_array = np.array(bw_img)

            # Resize the image (e.g., double the size)
            resized_img = cv2.resize(bw_img_array, (0,0), fx=10, fy=10, interpolation = cv2.INTER_CUBIC)

            # Contrast Enhancement
            clahe = cv2.createCLAHE(clipLimit=4, tileGridSize=(16, 16))
            enhanced_img = clahe.apply(resized_img)

            # Apply Otsu's thresholding
            _, thresh_img = cv2.threshold(enhanced_img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

            if label == CUSTOM_CLASSES["y-axis"]:
                y_axis_image = cropped_image

                result = reader.readtext(thresh_img)
                text_results = [item[1] for item in result]                            # Extract only the text
                text_found = ','.join(text_results)

                for (bbox, text, prob) in result:
                    bottom_y1 = bbox[1][1]  # Get the y-coordinate of one of the bottom points
                    bottom_y2 = bbox[2][1]  # Get the y-coordinate of the other bottom point
                    #print(bottom_y1, bottom_y2)
                    average_y = (bottom_y1 + bottom_y2) / 2  # Calculate the average y-coordinate
                    quarter_y = (bottom_y2 + average_y) / 2
                    yaxis_coordinates.append(int(quarter_y / 10))  # Add the average y-coordinate for each text box available (divide by the upscale factor)
            else:
                result = reader.readtext(thresh_img, detail = 0)      
                text_found = ','.join(result)
                text_found = re.sub('\n+', ',', text_found)                                           # Replace one or more consecutive newline characters with a single comma


            label_name = [name for name, id in CUSTOM_CLASSES.items() if id == label][0]              # Find the label name using the CUSTOM_CLASSES dictionary
            results[label_name] = text_found                                                          # Store the result in the dictionary

        elif label == 5 and score > 0.7:                                                              # Check if the label is 5 and the score is above 0.7
            xmin, ymin, xmax, ymax = [int(i) for i in box]                                            # Convert box coordinates to integers
            label5_image = img.crop((xmin, ymin, xmax, ymax))                                         # Crop the image for label 5

    # The results dictionary now contains the extracted text for labels 1 to 4
    #print(results)

    #############################################################################################################################
    # Apply the function to both lists
    try:
        y_axis_str = results['y-axis']
        print(f"Y axis string value is: {y_axis_str}")
        print("I am in 1st try")
        y_axis_values, y_axis_val_rem_idx = process_ocr_string(y_axis_str)
        print(f"Y axis string corrected value is: {y_axis_values}")

        yaxis_coordinates = [value for index, value in enumerate(yaxis_coordinates) if index not in y_axis_val_rem_idx]
        y_axis_values_new = calculate_new_list(y_axis_values)
        yaxis_coordinates_new = calculate_new_list(yaxis_coordinates)
        yaxis_coordinates_new = list(map(round, yaxis_coordinates_new))

        ######################################################################################################################
        # FIRST INTERPOLATION METHOD
        # Value interpolation from curve
        from scipy.interpolate import interp1d

        # Create a linear interpolation function
        f = interp1d(yaxis_coordinates_new, y_axis_values_new)

        ######################################################################################################################
        # SECOND INTERPOLATION METHOD
        y_axis_val_diff = abs(y_axis_values[-1] - y_axis_values[0])
        y_axis_coord_dif = abs(yaxis_coordinates[-1] - yaxis_coordinates[0])
        y_axis_val_per_pixel = y_axis_val_diff/y_axis_coord_dif
        
    except:
        continue
   #############################################################################################################################  
    
    # Plot processing
    bw_label5_img = label5_image.convert("L")                                                       # Convert the cropped image to grayscale (black and white)
    bw_label5_img_array = np.array(bw_label5_img)

    # Apply Gaussian blur to reduce noise and improve the result of Canny edge detection
    blurred_label5_img = cv2.GaussianBlur(bw_label5_img_array, (3, 3), 0)

    # First, apply a binary threshold on the image
    thresh_label5_img = cv2.adaptiveThreshold(blurred_label5_img, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 11, 2)
    inverted_label5_img = cv2.bitwise_not(thresh_label5_img)

    # Define a kernel for the erosion (a 3x3 square is common)
    kernel = np.ones((2,2), np.uint8)
    closed_label5_img = cv2.morphologyEx(inverted_label5_img, cv2.MORPH_CLOSE, kernel)

    # Find the contours
    contours, hierarchy = cv2.findContours(closed_label5_img, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Calculate the perimeter for each contour and find the index of the longest one
    max_length = 0
    longest_contour_index = 0
    for i, contour in enumerate(contours):
        length = cv2.arcLength(contour, True) # The 'True' argument indicates that the contour is closed
        if length > max_length:
            max_length = length
            longest_contour_index = i

    # Extract the longest contour
    longest_contour = contours[longest_contour_index]

    output_image2 = np.zeros_like(closed_label5_img)
    height, width = output_image2.shape
    filtered_coordinates = filter_coordinates(longest_contour)
    filtered_coordinates = remove_corner_pixels(filtered_coordinates, width, height)

    # Finding the highest pixel (smallest y value)
    highest_pixel = filtered_coordinates[filtered_coordinates[:, 1].argmin()]

    # Finding the lowest pixel (largest y value)
    lowest_pixel = filtered_coordinates[filtered_coordinates[:, 1].argmax()]

    # Finding the rightmost pixel (largest x value), and among those, the one with the lowest y value
    right_most_pixels = filtered_coordinates[filtered_coordinates[:, 0] == filtered_coordinates[:, 0].max()]
    right_most_pixel = right_most_pixels[right_most_pixels[:, 1].argmin()]


    ######################################################################################################################
    # this function finds the average difference between all the elements in a list, then adds a new element at the end
    # equal to the last element minus the average

    ######################################################################################################################
    try:
        print("I am in try")
        type1_interp_current_val = round(float(f(right_most_pixel[1])), 5)
        #print(f"The Y-axis value for the Y coordinate {right_most_pixel[1]} is {type1_interp_current_val}")

        # value from plot
        value_str = correct_value(correct_decimal_point(results['value']))
        stock_value = extract_float_value(value_str)
        #print(f"The stock value from the plot is {stock_value}")

        # value difference/accuracy
        type1_pct_diff = abs(round(((stock_value - type1_interp_current_val) / stock_value) * 100, 5))
        #print(f"The percentage difference is {type1_pct_diff:.2f}%")

        ######################################################################################################################
        # SECOND INTERPOLATION METHOD

        plot_height = label5_image.height
        type2_interp_current_val = round((abs(plot_height - right_most_pixel[1])) * y_axis_val_per_pixel, 5)

        #print(f"Curve value based on secondaly calculation is {type2_interp_current_val}")

        # value from plot
        type2_pct_diff = abs(round(((stock_value - type2_interp_current_val) / stock_value) * 100, 3))
        #print(f"The percentage difference is {type2_pct_diff:.3f}%")

        ######################################################################################################################
        first_algo_diff.append(type1_pct_diff)
        second_algo_diff.append(type2_pct_diff)
        
    except:
        continue
######################################################################################################################
def calculate_average(values):
    return sum(values) / len(values)

def remove_outliers(values):
    # Calculate Q1 and Q3
    Q1 = sorted(values)[int(len(values) * 0.25)]
    Q3 = sorted(values)[int(len(values) * 0.75)]
    
    # Calculate IQR
    IQR = Q3 - Q1
    
    # Define bounds for outliers
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Remove outliers
    return [value for value in values if lower_bound <= value <= upper_bound]
######################################################################################################################
print(first_algo_diff)
print(second_algo_diff)

# Calculate initial average
initial_avg_first_algo = calculate_average(first_algo_diff)
print(f"Initial Average for 1st Algo: {initial_avg_first_algo}")

# Remove outliers
filtered_values_first_algo = remove_outliers(first_algo_diff)
print(f"The number of processed images after outlier removal for 1st algo is: {filtered_values_first_algo}")

# Calculate average again
final_avg_first_algo_diff = calculate_average(filtered_values_first_algo)
print(f"Average after removing outliers for 1st Algo: {final_avg_first_algo_diff}")

######################################################################################################################
# Calculate initial average
initial_avg_second_algo = calculate_average(second_algo_diff)
print(f"Initial Average for 2nd Algo: {initial_avg_second_algo}")

# Remove outliers
filtered_values_second_algo = remove_outliers(second_algo_diff)
print(f"The number of processed images after outlier removal for 2nd algo is: {filtered_values_second_algo}")

# Calculate average again
final_avg_second_algo_diff = calculate_average(filtered_values_second_algo)
print(f"Average after removing outliers for 2nd Algo: {final_avg_second_algo_diff}")

Processing files:   0%|                                                               | 1/1000 [00:01<28:28,  1.71s/it]

Y axis string value is: 0.0035,0.003,0.0025,0.002,0.0015,0.001
I am in 1st try
Y axis string corrected value is: [0.0035, 0.003, 0.0025, 0.002, 0.0015, 0.001]
I am in try


Processing files:   0%|▏                                                              | 2/1000 [00:03<25:38,  1.54s/it]

Y axis string value is: 
I am in 1st try


Processing files:   0%|▏                                                              | 3/1000 [00:04<25:13,  1.52s/it]

...

Y axis string value is: 1.22,0.8,0.6:,0.4
I am in 1st try
Y axis string corrected value is: [1.22, 0.8, 0.4]
I am in try


Processing files: 100%|████████████████████████████████████████████████████████████▉| 998/1000 [25:07<00:03,  1.52s/it]

Y axis string value is: 
I am in 1st try


Processing files: 100%|████████████████████████████████████████████████████████████| 1000/1000 [25:10<00:00,  1.51s/it]

Y axis string value is: 
I am in 1st try
[0.07047, 180.33241, 131.56089, 1.70512, 0.22314, 3.68648, 1.2752, 100.0, 100.0, 7.69231, 3.29218, 163.73699, 0.22222, 37.45, 0.11503, 3926.35432, 1415.87629, 8.75912, 2.94842, 30.94137, 100.0, 100.0, 497.63932, 0.57995, 100.0, 152.18182, 19.47734, 9.09089, 0.14302, 99.99217, 100.0, 0.02966, 3955.66325, 100.0, 79.49616, 7.09645, 99.89973, 5.13241, 5707.08661, 1394.32158, 0.21518, 613.86575, 26.00806, 60.2122, 24.96876, 0.17009, 65983.07238, 7.00539, 191.26214, 7.58472, 0.04114, 100.0, 3.41262, 0.12323, 2374253127.98227, 4.5967, 100.0, 0.30398, 0.01657, 100.0, 9.2233, 0.0, 3.57513, 0.03968, 100.0, 100.0, 4.0024, 166.5618, 4.88492, 4.7619, 0.09773, 0.48492, 100.0, 497.8706, 325.94444, 0.03587, 0.08167, 100.0, 0.80429, 100.0, 2.71146, 1104.26065, 4.91939, 1.40845, 0.28948, 0.14038, 510.02045, 445.1762, 1.86301, 1.76817, 100.0, 100.0, 3.47263, 22.81579, 2.59707, 100.0, 100.0, 70.66943, 68.076, 100.0, 74.53311, 922442.84362, 100.0, 5.04463, 0.67797, 

In [39]:
from joblib import Parallel, delayed
import os
import random

# Specify the directory path
directory_path = 'pascal_voc_datasets/VOCdevkit/PlotsEnchanced_Original_NoAugmentation/JPEGImages/'

# Get all filenames from the directory
filenames = [f for f in os.listdir(directory_path) if os.path.isfile(os.path.join(directory_path, f))]

# Shuffle the filenames
random.shuffle(filenames)
#filenames = filenames[0:5]

def process_chunk(filenames_chunk):
    print(f"Processing chunk with {len(filenames_chunk)} filenames.")
    first_algo_diff_chunk = []
    second_algo_diff_chunk = []
    for img_name in filenames_chunk:
            CUSTOM_CLASSES = {"name": 1, "value": 2, "x-axis": 3, "y-axis": 4, "plot":5}

            img = Image.open(directory_path + img_name)
            orig_width, orig_height = img.size                                   # Store the original image dimensions

            # Process the image
            img_processed = data_transforms(img)

            # Calculate the scale ratios
            new_width, new_height = img_processed.shape[1], img_processed.shape[2]

            # Calculate the scale ratios
            if orig_width <= orig_height:
                new_height = 512
                new_width = int(orig_width * (new_height / orig_height))
            else:
                new_width = 512
                new_height = int(orig_height * (new_width / orig_width))

            width_ratio = new_width / orig_width
            height_ratio = new_height / orig_height

            img_processed = img_processed.to(device)
            with torch.no_grad():
                prediction = model([img_processed])

            # Scale bounding boxes back to original image size
            for box in prediction[0]['boxes']:
                box[0] *= (1 / width_ratio)
                box[2] *= (1 / width_ratio)
                box[1] *= (1 / height_ratio)
                box[3] *= (1 / height_ratio)


            results = {}                                 # Dictionary to store the results
            yaxis_coordinates = []                       # Initialize a list to store the Y coordinates
            label5_image = None                          # Variable to store the cropped image for label 5
            y_axis_image = None

            reader = easyocr.Reader(lang_list=['en'])       # Create a reader to recognize the English language

            for box, label, score in zip(prediction[0]['boxes'], prediction[0]['labels'], prediction[0]['scores']):
                if label >= 1 and label <= 4 and score > 0.6:                                                 # Check if the score is above 0.7 and label is between 1 and 4
                    xmin, ymin, xmax, ymax = [int(i) for i in box]                                            # Convert box coordinates to integers
                    cropped_image = img.crop((xmin, ymin, xmax, ymax))                                        # Crop the image          
                    bw_img = cropped_image.convert("L")                                                       # Convert the cropped image to grayscale (black and white)
                    bw_img_array = np.array(bw_img)

                    # Resize the image (e.g., double the size)
                    resized_img = cv2.resize(bw_img_array, (0,0), fx=10, fy=10, interpolation = cv2.INTER_CUBIC)

                    # Contrast Enhancement
                    clahe = cv2.createCLAHE(clipLimit=4, tileGridSize=(16, 16))
                    enhanced_img = clahe.apply(resized_img)

                    # Apply Otsu's thresholding
                    _, thresh_img = cv2.threshold(enhanced_img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

                    if label == CUSTOM_CLASSES["y-axis"]:
                        y_axis_image = cropped_image

                        result = reader.readtext(thresh_img)
                        text_results = [item[1] for item in result]                            # Extract only the text
                        text_found = ','.join(text_results)

                        for (bbox, text, prob) in result:
                            bottom_y1 = bbox[1][1]  # Get the y-coordinate of one of the bottom points
                            bottom_y2 = bbox[2][1]  # Get the y-coordinate of the other bottom point
                            #print(bottom_y1, bottom_y2)
                            average_y = (bottom_y1 + bottom_y2) / 2  # Calculate the average y-coordinate
                            quarter_y = (bottom_y2 + average_y) / 2
                            yaxis_coordinates.append(int(quarter_y / 10))  # Add the average y-coordinate for each text box available (divide by the upscale factor)
                    else:
                        result = reader.readtext(thresh_img, detail = 0)      
                        text_found = ','.join(result)
                        text_found = re.sub('\n+', ',', text_found)                                           # Replace one or more consecutive newline characters with a single comma


                    label_name = [name for name, id in CUSTOM_CLASSES.items() if id == label][0]              # Find the label name using the CUSTOM_CLASSES dictionary
                    results[label_name] = text_found                                                          # Store the result in the dictionary

                elif label == 5 and score > 0.7:                                                              # Check if the label is 5 and the score is above 0.7
                    xmin, ymin, xmax, ymax = [int(i) for i in box]                                            # Convert box coordinates to integers
                    label5_image = img.crop((xmin, ymin, xmax, ymax))                                         # Crop the image for label 5

            # The results dictionary now contains the extracted text for labels 1 to 4
            #print(results)

            #############################################################################################################################
            # Apply the function to both lists
            try:
                y_axis_str = results['y-axis']
                print(f"Y axis string value is: {y_axis_str}")
                print("I am in 1st try")
                y_axis_values, y_axis_val_rem_idx = process_ocr_string(y_axis_str)
                print(f"Y axis string corrected value is: {y_axis_values}")

                yaxis_coordinates = [value for index, value in enumerate(yaxis_coordinates) if index not in y_axis_val_rem_idx]
                y_axis_values_new = calculate_new_list(y_axis_values)
                yaxis_coordinates_new = calculate_new_list(yaxis_coordinates)
                yaxis_coordinates_new = list(map(round, yaxis_coordinates_new))

                ######################################################################################################################
                # FIRST INTERPOLATION METHOD
                # Value interpolation from curve
                from scipy.interpolate import interp1d

                # Create a linear interpolation function
                f = interp1d(yaxis_coordinates_new, y_axis_values_new)

                ######################################################################################################################
                # SECOND INTERPOLATION METHOD
                y_axis_val_diff = abs(y_axis_values[-1] - y_axis_values[0])
                y_axis_coord_dif = abs(yaxis_coordinates[-1] - yaxis_coordinates[0])
                y_axis_val_per_pixel = y_axis_val_diff/y_axis_coord_dif

            except:
                continue
           #############################################################################################################################  

            # Plot processing
            bw_label5_img = label5_image.convert("L")                                                       # Convert the cropped image to grayscale (black and white)
            bw_label5_img_array = np.array(bw_label5_img)

            # Apply Gaussian blur to reduce noise and improve the result of Canny edge detection
            blurred_label5_img = cv2.GaussianBlur(bw_label5_img_array, (3, 3), 0)

            # First, apply a binary threshold on the image
            thresh_label5_img = cv2.adaptiveThreshold(blurred_label5_img, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 11, 2)
            inverted_label5_img = cv2.bitwise_not(thresh_label5_img)

            # Define a kernel for the erosion (a 3x3 square is common)
            kernel = np.ones((2,2), np.uint8)
            closed_label5_img = cv2.morphologyEx(inverted_label5_img, cv2.MORPH_CLOSE, kernel)

            # Find the contours
            contours, hierarchy = cv2.findContours(closed_label5_img, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

            # Calculate the perimeter for each contour and find the index of the longest one
            max_length = 0
            longest_contour_index = 0
            for i, contour in enumerate(contours):
                length = cv2.arcLength(contour, True) # The 'True' argument indicates that the contour is closed
                if length > max_length:
                    max_length = length
                    longest_contour_index = i

            # Extract the longest contour
            longest_contour = contours[longest_contour_index]

            output_image2 = np.zeros_like(closed_label5_img)
            height, width = output_image2.shape
            filtered_coordinates = filter_coordinates(longest_contour)
            filtered_coordinates = remove_corner_pixels(filtered_coordinates, width, height)

            # Finding the highest pixel (smallest y value)
            highest_pixel = filtered_coordinates[filtered_coordinates[:, 1].argmin()]

            # Finding the lowest pixel (largest y value)
            lowest_pixel = filtered_coordinates[filtered_coordinates[:, 1].argmax()]

            # Finding the rightmost pixel (largest x value), and among those, the one with the lowest y value
            right_most_pixels = filtered_coordinates[filtered_coordinates[:, 0] == filtered_coordinates[:, 0].max()]
            right_most_pixel = right_most_pixels[right_most_pixels[:, 1].argmin()]


            ######################################################################################################################
            # this function finds the average difference between all the elements in a list, then adds a new element at the end
            # equal to the last element minus the average

            ######################################################################################################################
            try:
                print("I am in try")
                type1_interp_current_val = round(float(f(right_most_pixel[1])), 5)
                #print(f"The Y-axis value for the Y coordinate {right_most_pixel[1]} is {type1_interp_current_val}")

                # value from plot
                value_str = correct_value(correct_decimal_point(results['value']))
                stock_value = extract_float_value(value_str)
                #print(f"The stock value from the plot is {stock_value}")

                # value difference/accuracy
                type1_pct_diff = abs(round(((stock_value - type1_interp_current_val) / stock_value) * 100, 5))
                #print(f"The percentage difference is {type1_pct_diff:.2f}%")

                ######################################################################################################################
                # SECOND INTERPOLATION METHOD

                plot_height = label5_image.height
                type2_interp_current_val = round((abs(plot_height - right_most_pixel[1])) * y_axis_val_per_pixel, 5)

                #print(f"Curve value based on secondaly calculation is {type2_interp_current_val}")

                # value from plot
                type2_pct_diff = abs(round(((stock_value - type2_interp_current_val) / stock_value) * 100, 3))
                #print(f"The percentage difference is {type2_pct_diff:.3f}%")

                ######################################################################################################################
                return first_algo_diff_chunk, second_algo_diff_chunk

            except:
                continue

                
                                
# Split the filenames into chunks
n_jobs = -1  # Use all available CPUs
chunk_size = len(filenames) // abs(n_jobs)
filenames_chunks = [filenames[i:i + chunk_size] for i in range(0, len(filenames), chunk_size)]

first_chunk_results = process_chunk(filenames_chunks[0])
print(first_chunk_results)                


# # Use joblib to process the chunks in parallel and tqdm to show progress
# results = Parallel(n_jobs=n_jobs)(delayed(process_chunk)(chunk) for chunk in tqdm(filenames_chunks, desc="Processing files"))

# # Combine the results from all chunks
# first_algo_diff = [item for sublist in [res[0] for res in results] for item in sublist]
# second_algo_diff = [item for sublist in [res[1] for res in results] for item in sublist]



# ######################################################################################################################
# def calculate_average(values):
#     return sum(values) / len(values)

# def remove_outliers(values):
#     # Calculate Q1 and Q3
#     Q1 = sorted(values)[int(len(values) * 0.25)]
#     Q3 = sorted(values)[int(len(values) * 0.75)]
    
#     # Calculate IQR
#     IQR = Q3 - Q1
    
#     # Define bounds for outliers
#     lower_bound = Q1 - 1.5 * IQR
#     upper_bound = Q3 + 1.5 * IQR
    
#     # Remove outliers
#     return [value for value in values if lower_bound <= value <= upper_bound]
# ######################################################################################################################
# try:
#     print(first_algo_diff)
#     print(second_algo_diff)

#     # Calculate initial average
#     initial_avg_first_algo = calculate_average(first_algo_diff)
#     print(f"Initial Average for 1st Algo: {initial_avg_first_algo}")

#     # Remove outliers
#     filtered_values_first_algo = remove_outliers(first_algo_diff)
#     print(f"The number of processed images after outlier removal for 1st algo is: {filtered_values_first_algo}")

#     # Calculate average again
#     final_avg_first_algo_diff = calculate_average(filtered_values_first_algo)
#     print(f"Average after removing outliers for 1st Algo: {final_avg_first_algo_diff}")

#     ######################################################################################################################
#     # Calculate initial average
#     initial_avg_second_algo = calculate_average(second_algo_diff)
#     print(f"Initial Average for 2nd Algo: {initial_avg_second_algo}")

#     # Remove outliers
#     filtered_values_second_algo = remove_outliers(second_algo_diff)
#     print(f"The number of processed images after outlier removal for 2nd algo is: {filtered_values_second_algo}")

#     # Calculate average again
#     final_avg_second_algo_diff = calculate_average(filtered_values_second_algo)
#     print(f"Average after removing outliers for 2nd Algo: {final_avg_second_algo_diff}")
# except:
#     pass

Processing chunk with 144889 filenames.
Y axis string value is: 0.0047,0.0047,0.0046,0.0046,0.0045,0.0045,0.0044,0.0044,0.0043,0.0043,0.0042,00042,0.0041,",0.0041,0.0040,0.0040,0.0039,0.0039,0.0038,0.0038,0.0037,0.0037,Cop
I am in 1st try
Y axis string corrected value is: [0.0047, 0.0047, 0.0046, 0.0046, 0.0045, 0.0045, 0.0044, 0.0044, 0.0043, 0.0043, 0.0042, 0.0042, 0.0041, 0.0041, 0.004, 0.004, 0.0039, 0.0039, 0.0038, 0.0038, 0.0037, 0.0037]
I am in try
([], [])


In [ ]:
# KERAS-OCR TEST 
image_path = 'pascal_voc_datasets/VOCdevkit/PlotsEnchanced_Original_NoAugmentation/JPEGImages/'
img_name = 'image117687'
CUSTOM_CLASSES = {"name": 1, "value": 2, "x-axis": 3, "y-axis": 4, "plot":5}

img = Image.open(image_path + img_name + '.jpg')
orig_width, orig_height = img.size                                   # Store the original image dimensions

# Process the image
img_processed = data_transforms(img)

# Calculate the scale ratios
new_width, new_height = img_processed.shape[1], img_processed.shape[2]

# Calculate the scale ratios
if orig_width <= orig_height:
    new_height = 512
    new_width = int(orig_width * (new_height / orig_height))
else:
    new_width = 512
    new_height = int(orig_height * (new_width / orig_width))
    
width_ratio = new_width / orig_width
height_ratio = new_height / orig_height

with torch.no_grad():
    prediction = model([img_processed])

# Scale bounding boxes back to original image size
for box in prediction[0]['boxes']:
    box[0] *= (1 / width_ratio)
    box[2] *= (1 / width_ratio)
    box[1] *= (1 / height_ratio)
    box[3] *= (1 / height_ratio)

# Debugging
#print(prediction[0]['boxes'])
print(prediction[0]['scores'])


# Create a recognition pipeline
pipeline = keras_ocr.pipeline.Pipeline()

results = {}                                 # Dictionary to store the results
label5_image = None                          # Variable to store the cropped image for label 5

for box, label, score in zip(prediction[0]['boxes'], prediction[0]['labels'], prediction[0]['scores']):
    if label >= 1 and label <= 4 and score > 0.7:
        xmin, ymin, xmax, ymax = [int(i) for i in box]
        cropped_image = img.crop((xmin, ymin, xmax, ymax))
        cropped_image_array = np.array(cropped_image)

        plt.figure(figsize=(10, 10))
        plt.imshow(thresh_img, cmap='gray')
        plt.title('Image')
        plt.axis('off')
        plt.show()

        result = pipeline.recognize([cropped_image_array])[0]
        text = ','.join([word[0] for word in result])
        text = re.sub('\n+', ',', text)
        label_name = [name for name, id in CUSTOM_CLASSES.items() if id == label][0]
        results[label_name] = text
        
    elif label == 5 and score > 0.8:
        xmin, ymin, xmax, ymax = [int(i) for i in box]
        label5_image = img.crop((xmin, ymin, xmax, ymax))

print(results)

In [ ]:
# PYTESSERACT OCR TESTING
image_path = 'pascal_voc_datasets/VOCdevkit/PlotsEnchanced_Original_NoAugmentation/JPEGImages/'
img_name = 'image1'
CUSTOM_CLASSES = {"name": 1, "value": 2, "x-axis": 3, "y-axis": 4, "plot":5}

img = Image.open(image_path + img_name + '.jpg')
orig_width, orig_height = img.size                                   # Store the original image dimensions

# Process the image
img_processed = data_transforms(img)

# Calculate the scale ratios
new_width, new_height = img_processed.shape[1], img_processed.shape[2]

# Calculate the scale ratios
if orig_width <= orig_height:
    new_height = 512
    new_width = int(orig_width * (new_height / orig_height))
else:
    new_width = 512
    new_height = int(orig_height * (new_width / orig_width))
    
width_ratio = new_width / orig_width
height_ratio = new_height / orig_height

with torch.no_grad():
    prediction = model([img_processed])

# Scale bounding boxes back to original image size
for box in prediction[0]['boxes']:
    box[0] *= (1 / width_ratio)
    box[2] *= (1 / width_ratio)
    box[1] *= (1 / height_ratio)
    box[3] *= (1 / height_ratio)

# Debugging
#print(prediction[0]['boxes'])
print(prediction[0]['scores'])


results = {}                                 # Dictionary to store the results
label5_image = None                          # Variable to store the cropped image for label 5

for box, label, score in zip(prediction[0]['boxes'], prediction[0]['labels'], prediction[0]['scores']):
    if label >= 1 and label <= 4 and score > 0.7:                                                 # Check if the score is above 0.7 and label is between 1 and 4
        xmin, ymin, xmax, ymax = [int(i) for i in box]                                            # Convert box coordinates to integers
        cropped_image = img.crop((xmin, ymin, xmax, ymax))                                        # Crop the image
        bw_img = cropped_image.convert("L")                                                       # Convert the cropped image to grayscale (black and white)
        bw_img_array = np.array(bw_img)

        # Resize the image (e.g., double the size)
        resized_img = cv2.resize(bw_img_array, (0,0), fx=2, fy=2, interpolation = cv2.INTER_CUBIC)

        # Contrast Enhancement
        clahe = cv2.createCLAHE(clipLimit=1.5, tileGridSize=(32, 32))
        enhanced_img = clahe.apply(resized_img)

        # Apply Otsu's thresholding
        _, thresh_img = cv2.threshold(enhanced_img, 200, 30, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        
        # Assuming resized_img has been defined as in your snippet
        plt.figure(figsize=(10, 10))
        plt.imshow(thresh_img, cmap='gray')
        plt.title('Image')
        plt.axis('off')  # To turn off the axis
        plt.show()


        
        # Determine the whitelist of characters based on the label
        if label == CUSTOM_CLASSES['y-axis']:
            config = r"--oem 1 --psm 3 -c tessedit_char_whitelist=0123456789.e-"
        else:
            config = '--oem 1'
            
        text = pytesseract.image_to_string(thresh_img, config=config)                             # Process the cropped image with Pytesseract
        text = re.sub('\n+', ',', text)                                                           # Replace one or more consecutive newline characters with a single comma
        label_name = [name for name, id in CUSTOM_CLASSES.items() if id == label][0]              # Find the label name using the CUSTOM_CLASSES dictionary
        results[label_name] = text                                                                # Store the result in the dictionary
        
    elif label == 5 and score > 0.8:                                                              # Check if the label is 5 and the score is above 0.7
        xmin, ymin, xmax, ymax = [int(i) for i in box]                                            # Convert box coordinates to integers
        label5_image = img.crop((xmin, ymin, xmax, ymax))                                         # Crop the image for label 5

# The results dictionary now contains the extracted text for labels 1 to 4
print(results)



In [ ]:
# Draw the bounding boxes on the image
fig, ax = plt.subplots(1, figsize=(17, 15))  # Adjust the size as needed. Here, the figure size is 12x12 inches.
ax.imshow(img)

colors = ['r', 'g', 'b', 'y', 'm']

# We will draw all the boxes with a score above 0.7
for box, label, score in zip(prediction[0]['boxes'], prediction[0]['labels'], prediction[0]['scores']):
    if score > 0.7:  # Check if the score is above 0.9
        xmin, ymin, xmax, ymax = box.numpy()  # Move the box coordinates to the CPU
        rect = patches.Rectangle((xmin, ymin), (xmax-xmin), (ymax-ymin), edgecolor=colors[label%4], facecolor='none')
        ax.add_patch(rect)

plt.show()

In [ ]:
from sklearn.linear_model import LinearRegression

# Create a linear regression model and fit it to the data
yaxis_coordinates_new_array = np.array(yaxis_coordinates_new).reshape(-1, 1)
y_axis_values_new_array = np.array(y_axis_values_new)

lr_model = LinearRegression()
lr_model.fit(yaxis_coordinates_new_array, y_axis_values_new_array)

# Define a function to get the Y-axis value for a given Y coordinate
def get_y_axis_value(y_coord):
    return lr_model.predict([[y_coord]])[0]

new_y_coord = 348
print(f"The Y-axis value for the Y coordinate {new_y_coord} is {get_y_axis_value(new_y_coord)}")